In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
print(torch.cuda.device_count())

1


In [3]:
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [4]:
import cv2

from ultralytics import YOLO

In [12]:


from ultralytics import YOLO
import numpy as np

def estimate_distance_in_meters(xyxy, focal_length=350, real_height=1.5):
    """Burada kamera focal lengthi 350 ve arabanın boyunun 1.5 metre olacak şekilde kameraya olan yaklaşık kadraj uzunluğu ölçülüyor"""
    height_in_pixels = xyxy[3] - xyxy[1]
    distance_meters = (real_height * focal_length) / height_in_pixels
    return distance_meters

def process_video_with_distance_estimation(input_video_path, output_video_path):
    # yolo v8 modeli yüklüyorum.Bilgisayarın cpu kullanımından kısıp gpu kullanmasını sağlıyorum cuda sayesinde
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = YOLO('yolov8n.pt').to(device) 

    # input video pathi açmak
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {input_video_path}")
        return

    # input videonun genişlik ve uzunluk çözünürlüklerini alıyorum.
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # output olarak mp4 video çıktısı istediğim için videowriter kullanıyorum
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO modeli ile object detection işlemi
        results = model(frame, device=device)

        # çıktıdaki her bir box un distanceını hesaplıyorum
        for result in results:
            for box in result.boxes:
                class_id = int(box.cls[0])
                xyxy = box.xyxy[0].cpu().numpy().astype(int)

                if class_id == 2:  # yalnızca arabaları bulması için modeldeki car parametresine denk gelen class Id seçiyorum
                    # distancelerı hesapladığım kod
                    distance_estimate = estimate_distance_in_meters(xyxy)
                    distance_label = f'{distance_estimate:.2f} meters'

                    # Bounding boxları videoda göstermek için rectangel shape çiziyorum ve distance texti koyuyorum
                    cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (255, 0, 0), 2)
                    cv2.putText(frame, distance_label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # kaydettiğim her bir frame i output videoya yazıyorum
        out.write(frame)

        # Model çalışırken frameleri anlık görebilmek için imshow komutunu kullanıyorum
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

















In [ ]:

process_video_with_distance_estimation('myvideo6.mp4', 'output_video2.mp4')


0: 384x640 7 cars, 34.0ms
Speed: 5.0ms preprocess, 34.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 30.0ms
Speed: 5.0ms preprocess, 30.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 30.0ms
Speed: 4.0ms preprocess, 30.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 27.0ms
Speed: 4.0ms preprocess, 27.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 27.0ms
Speed: 4.0ms preprocess, 27.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 26.0ms
Speed: 5.0ms preprocess, 26.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 27.0ms
Speed: 4.0ms preprocess, 27.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 27.0ms
Speed: 4.0ms preprocess, 27.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384